# create repackaged WOA DATA for each Dataset

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob

In [2]:
mld1 = xr.open_dataset('MLDClimatology_DeBoyerMontagut/mld_DT02_c1m_reg2.0.nc')

/Users/bpo/anaconda/anaconda3/lib/python3.7/site-packages/xarray/coding/times.py:465: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/bpo/anaconda/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


In [67]:
mld1.variables

Frozen(OrderedDict([('krig_std_dev', <xarray.Variable (time: 12, lat: 90, lon: 180)>
[194400 values with dtype=float32]
Attributes:
    units:       meters
    long_name:   statistical std deviation linked to kriging of mld
    min_value:   0.0
    max_value:   213.17142
    mask_value:  1000000000.0), ('lat', <xarray.IndexVariable 'lat' (lat: 90)>
array([-88. , -86. , -84. , -82. , -80. , -78. , -76. , -74. , -72. , -70. ,
       -68. , -66. , -64. , -62. , -60. , -58. , -56. , -54. , -52. , -50. ,
       -48. , -46. , -44. , -42. , -40. , -38. , -36. , -34. , -32. , -30. ,
       -28. , -26. , -24. , -22. , -20. , -18. , -16. , -14. , -12. , -10. ,
        -8. ,  -6. ,  -4. ,  -2. ,   0. ,   2. ,   4. ,   6. ,   8. ,  10. ,
        12. ,  14. ,  16. ,  18. ,  20. ,  22. ,  24. ,  26. ,  28. ,  30. ,
        32. ,  34. ,  36. ,  38. ,  40. ,  42. ,  44. ,  46. ,  48. ,  50. ,
        52. ,  54. ,  56. ,  58. ,  60. ,  62. ,  64. ,  66. ,  68. ,  70. ,
        72. ,  74. ,  76. ,  78. 

In [65]:
WOAtest.variables

Frozen(OrderedDict([('crs', <xarray.Variable ()>
array(-2147483647, dtype=int32)
Attributes:
    grid_mapping_name:            latitude_longitude
    epsg_code:                    EPSG:4326
    longitude_of_prime_meridian:  0.0
    semi_major_axis:              6378137.0
    inverse_flattening:           298.25723), ('lat', <xarray.IndexVariable 'lat' (lat: 180)>
array([-89.5, -88.5, -87.5, -86.5, -85.5, -84.5, -83.5, -82.5, -81.5, -80.5,
       -79.5, -78.5, -77.5, -76.5, -75.5, -74.5, -73.5, -72.5, -71.5, -70.5,
       -69.5, -68.5, -67.5, -66.5, -65.5, -64.5, -63.5, -62.5, -61.5, -60.5,
       -59.5, -58.5, -57.5, -56.5, -55.5, -54.5, -53.5, -52.5, -51.5, -50.5,
       -49.5, -48.5, -47.5, -46.5, -45.5, -44.5, -43.5, -42.5, -41.5, -40.5,
       -39.5, -38.5, -37.5, -36.5, -35.5, -34.5, -33.5, -32.5, -31.5, -30.5,
       -29.5, -28.5, -27.5, -26.5, -25.5, -24.5, -23.5, -22.5, -21.5, -20.5,
       -19.5, -18.5, -17.5, -16.5, -15.5, -14.5, -13.5, -12.5, -11.5, -10.5,
        -9.5,  -8.

In [46]:
WOAtemp.dims

Frozen(SortedKeysDict({'depth': 57, 'lon': 360, 'lat': 180, 'time': 12, 'nbounds': 2}))

In [3]:
TempNCDFfiles = glob.glob('WOA2018/Temperature/*.nc', recursive=True)
WOAtemp = xr.open_mfdataset(TempNCDFfiles,combine='by_coords',decode_times=False)

In [134]:
def MLD_at_latlon(lats=40,lons=-20, mlddata=mld1, justMLD=True):
    """This function returns MLD depth across the year at a specific location"""
    #ToDO include check whether the spot is on land or water
    #Make sure the lat lon usage across everything is coherent!
    mld2 = mlddata.sel(lat=lats,lon=lons,method='nearest').to_dataframe()
    mld2['time'] = mld2.index
    mld2.index = range(1,13)
    mld2.index.names = ['time']
    if mld2['mask'].any()==0:
        print('step1')
        return pd.DataFrame()
    elif justMLD==True:
        return mld2[['mld']]#.reset_index()
    else:
        return mld2#.reset_index()

In [135]:
a=MLD_at_latlon(20,10)
a.empty

step1


True

In [120]:
def WOADatInLocation(lats=40,lons=-20, WOAtempdata=WOAtemp,var='t_an', justTempDepth=True):
    """This function returns the xarray of Temperature in a specific location"""
    WOAtemp1 = WOAtempdata.sel(nbounds=0)
    WOAtemp_df = WOAtemp1.sel(lat=lats,lon=lons, method='nearest').to_dataframe()
    WOAtemp_df.index = pd.MultiIndex.from_arrays([WOAtemp_df.index.get_level_values(level='depth'), [i for j in range(0,57) for i in range(1,13)]])
    WOAtemp_df.index.names = ['depth','time']
    
    if justTempDepth==True:
        return WOAtemp_df[[var]]#.reset_index()
    else:
        return WOAtemp_df#.reset_index()

In [121]:
WOADatInLocation(20,10)

t_an
depth  time      
0.0    1      NaN
       2      NaN
       3      NaN
       4      NaN
       5      NaN
...           ...
1500.0 8      NaN
       9      NaN
       10     NaN
       11     NaN
       12     NaN

[684 rows x 1 columns]

In [122]:
def interpolateWOAprofiles(WOAdat):
    """this function interpolates the WOA profiles to a resolution of 1 meter"""
    WOAgrid = WOAdat.unstack(level=1)
    WOAgrid1 = WOAgrid.reindex(range(0,1501))
    return WOAgrid1.interpolate(method='linear')

In [123]:
def MeanAboveMLD(WOAint, MLD):
    """this function filters all temperature measurements above a certain depth and takes the mean"""
    WOAint_d1 = WOAint
    WOAint_d1.columns = WOAint.columns.droplevel()
    WOAx = WOAint_d1.reset_index()
    out = []
    for i in range(1,13):
        out.append({'Month':i,'Temp':WOAx[WOAx['depth'] < MLD[MLD['time']==i]['mld'].values[0]][i].mean()})
    return pd.DataFrame(out)

In [132]:
def MeanAboveMLDatLoc(lats=40, lons=-20, month = 1, mlddat=mld1, WOAdat=WOAtemp, var='t_an'):
    
    MLD = MLD_at_latlon(lats,lons,mlddat).reset_index()
    print('hi')
    if MLD.empty == True:
        print('yes')
        return True
    else:
        WOA = WOADatInLocation(lats,lons,WOAdat,var)
        WOA_int = interpolateWOAprofiles(WOA)
        return MeanAboveMLD(WOA_int,MLD)

In [133]:
MeanAboveMLDatLoc(20,10,7)

hi


,Month,Temp
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN
5,6,NaN
6,7,NaN
7,8,NaN
8,9,NaN
9,10,NaN


In [10]:
np.arange(0,12,1)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

# START CREATING NCDF FILE

In [27]:
from netCDF4 import Dataset

In [28]:
rootgrp = Dataset('13_netcdf4.nc', 'w', format='NETCDF4')

import time
rootgrp.description = 'WOA 2018 Temperature averaged above MLD'
rootgrp.history = "Created " + time.ctime(time.time())
rootgrp.source = "WOA2018 + DeBoyer Montague et al. 2004 MLD"

#nutgrp = rootgrp.createGroup("MLD_DeBoyer2004")
#tempgrp = rootgrp.createGroup("WOA2018")
#print(rootgrp.groups)

In [29]:
lats = np.arange(-90,91,1)
lons = np.arange(-180,180,1)
times = np.arange(1,13,1)

#[[i,j,k,MeanAboveMLDatLoc(i,j,k)] for k in time for i in lats[1:5] for j in lons[1:5]]

In [30]:
var = rootgrp.createDimension("var", None)
time = rootgrp.createDimension("time", len(times))
lat = rootgrp.createDimension("lat", len(lats))
lon = rootgrp.createDimension("lon", len(lons))
print(rootgrp.dimensions)

OrderedDict([('var', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'var', size = 0
), ('time', <class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 12
), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 181
), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 360
)])


In [31]:
print(rootgrp.groups)

OrderedDict()


In [32]:
varss = rootgrp.createVariable("var","f8",("lat","lon","time"))
month = rootgrp.createVariable("time","i4",("time",))
latitudes = rootgrp.createVariable("lat","f4",("lat",))
longitudes = rootgrp.createVariable("lon","f4",("lon",))

# two dimensions unlimited

print(varss)

<class 'netCDF4._netCDF4.Variable'>
float64 var(lat, lon, time)
unlimited dimensions: 
current shape = (181, 360, 12)
filling on, default _FillValue of 9.969209968386869e+36 used



In [33]:
latitudes.units = "degrees north"
longitudes.units = "degrees east"

month.units = "climatological months"
month.calendar = "gregorian"

print(rootgrp.__dict__)

OrderedDict([('description', 'WOA 2018 Temperature averaged above MLD'), ('history', 'Created Fri Nov  8 14:09:15 2019'), ('source', 'WOA2018 + DeBoyer Montague et al. 2004 MLD')])


In [34]:
latitudes[:] = lats
longitudes[:] = lons
month[:] = times

nlats = len(rootgrp.dimensions["lat"])
nlons = len(rootgrp.dimensions["lon"])
print("temp shape before adding data = {}".format(varss.shape))

temp shape before adding data = (181, 360, 12)


In [35]:
for i in range(len(lats[1:5])):
    
    for j in range(len(lons[1:5])):
        
        MAMLD = MeanAboveMLDatLoc(i,j)
        
        for k in range(len(times)):
            if MAMLD == True:
                varss[i, j, k] = np.nan
            else:
            varss[i, j, k] = MAMLD['Temp'].values[k]

varss

<class 'netCDF4._netCDF4.Variable'>
float64 var(lat, lon, time)
unlimited dimensions: 
current shape = (181, 360, 12)
filling on, default _FillValue of 9.969209968386869e+36 used

In [36]:
rootgrp.close()

In [96]:
np.nan == 0


False

In [41]:
testdata = xr.open_dataset('13_netcdf4.nc')
testdata.variables

Frozen(OrderedDict([('var', <xarray.Variable (lat: 181, lon: 360, time: 12)>
[781920 values with dtype=float64]), ('time', <xarray.IndexVariable 'time' (time: 12)>
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int32)
Attributes:
    units:     climatological months
    calendar:  gregorian), ('lat', <xarray.IndexVariable 'lat' (lat: 181)>
array([-90., -89., -88., -87., -86., -85., -84., -83., -82., -81., -80., -79.,
       -78., -77., -76., -75., -74., -73., -72., -71., -70., -69., -68., -67.,
       -66., -65., -64., -63., -62., -61., -60., -59., -58., -57., -56., -55.,
       -54., -53., -52., -51., -50., -49., -48., -47., -46., -45., -44., -43.,
       -42., -41., -40., -39., -38., -37., -36., -35., -34., -33., -32., -31.,
       -30., -29., -28., -27., -26., -25., -24., -23., -22., -21., -20., -19.,
       -18., -17., -16., -15., -14., -13., -12., -11., -10.,  -9.,  -8.,  -7.,
        -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,
         6.